In [54]:
#Let us start off by anonottating dumy transactions
transaction1 = {
  'amount': '30',
  'sender': 'Alice',
  'receiver': 'Bob'}
transaction2 = { 
  'amount': '200',
  'sender': 'Bob',
  'receiver': 'Alice'}
transaction3 = { 
  'amount': '300',
  'sender': 'Alice',
  'receiver': 'Timothy' }
transaction4 = { 
  'amount': '300',
  'sender': 'Rodrigo',
  'receiver': 'Thomas' }
transaction5 = { 
  'amount': '200',
  'sender': 'Timothy',
  'receiver': 'Thomas' }
transaction6 = { 
  'amount': '400',
  'sender': 'Tiffany',
  'receiver': 'Xavier' }

mempool = [transaction1, transaction2, transaction3, transaction4, transaction5, transaction6]

In [55]:
#Let us enable the user to generate transactions as much as they want and append them to the mempool
import names
import random
total_dummy_transactions = int(input("Please enter how many random transactions you want to generate: "))
senders = [names.get_first_name() for i in range(total_dummy_transactions)]
receivers = [names.get_first_name() for i in range(total_dummy_transactions)]
dummy_values = [random.randint(200,1500) for i in range(total_dummy_transactions)]
dummy_transactions = [
    {
        'amount':f"{dummy_values[i]}",
        'sender':f"{senders[i]}",
        'receiver':f"{receivers[i]}"    
    }
    for i in range(total_dummy_transactions)
]
mempool.extend(dummy_transactions)

Please enter how many random transactions you want to generate: 10000


In [56]:
class Validator:
    def __init__(self, pub_key, chain):
        self.pub_key = pub_key
        self.chain = chain
        self.consensus_code = 0
        self.validators_copy = [] #Contain public key of all validators
    def calculate_kwm(self,own=True):
        alphabets = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
        dictionary = {}
        i = 11
        j = 37
        for letter in alphabets:
            dictionary[letter.lower()] = j
            dictionary[letter] = i
            j += 1
            i+=1
        for i in range(10):
            dictionary[i] = i
        kwm = 0
        if own:
            hash_to_calculate = self.pub_key
        for individual_letter in hash_to_calculate:
            try:
                kwm += dictionary[individual_letter]
            except:
                kwm += dictionary[int(individual_letter)]
        return kwm

In [57]:
#The creation of the block
from datetime import datetime
from hashlib import sha256
class Block:
    #Initializing the block
    def __init__(self, transactions, previous_hash,nonce=0):
        self.transactions = transactions
        self.previous_hash = previous_hash
        self.nonce = nonce
        self.timestamp = datetime.now()
        #Call the geretate_hash to get the hash of block
        self.hash = self.generate_hash()
    def display_block_info(self):
        print("tmestamp: ", self.timestamp)
        print("transactions: ", self.transactions)
        print("Current hash: ", self.generate_hash())
    #We now proceed to generate hash for the block
    def generate_hash(self):
        block_contents = str(self.timestamp)+str(self.transactions)+str(self.nonce)+str(self.previous_hash)
        block_hash = sha256(block_contents.encode())
        return block_hash.hexdigest()

In [58]:
class Blockchain:
    def __init__(self):
        self.chain = []
        self.all_transactions = []
        self.genesis_block()
    
    def genesis_block(self):
        transactions = {}
        genesis_block = Block(transactions, "0")
        self.chain.append(genesis_block)
        return self.chain
    def print_blocks(self):
        for i in range(len(self.chain)):
            current_block = self.chain[i]
            print("Block {} {}".format(i, current_block))
            current_block.display_block_info()
    #Add block to the chain
    def add_block(self, transactions):
        self.previous_block_hash = self.chain[len(self.chain)-1].hash
        new_block = Block(transactions, self.previous_block_hash)
        #We need to calculate the proof of work
        proof = self.proof_of_work(new_block, difficulty=0)
        self.chain.append(new_block)
        return proof, new_block
    def validate_chain(self):
        for i in range(1, len(self.chain)):
            current = self.chain[i]
            previous = self.chain[i-1]
            if (current.hash != current.generate_hash()):
                print("The current hash of the block does not equal the generated hash of the block.")
                return False
            if (current.previous_hash != previous.generate_hash()):
                print("Previous block not equal previous hash value stored in current block")
                return False
        return True
    def proof_of_work(self,block,difficulty=2):
        proof = block.generate_hash()
        while proof[:difficulty] != '0'*difficulty:
            block.nonce += 1
            proof = block.generate_hash()
        block.nonce = 0
        return proof

In [59]:
#Staring off the tree chain
validator1_copy = Validator("axqPe96aiwZjQ", Blockchain())  
validator2_copy = Validator("aQfx12ijAtcTM",Blockchain())
validator3_copy = Validator("J94Vswa72liac", Blockchain())

In [60]:
#Let's now randomize the mempool
import math
import random
random.shuffle(mempool)

In [61]:
validators = [validator1_copy, validator2_copy, validator3_copy]
validators = sorted(validators, key=Validator.calculate_kwm, reverse=True)

In [62]:
#Assign the consensus code
consensus_reference = [1,2,3,4,5,6,7,8,9, 'A', 'B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U'
                      'V','W','X','Y','Z','a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','q','r','s','t','u','v',
                      'w','x','y','z']
code_multiplier = math.ceil(len(mempool)/len(consensus_reference))
consensus_codes = [str(consensus_reference[i])*code_multiplier for i in range(len(validators))]
#Assign the consensus_code to validators
for i in range(len(validators)):
    validators[i].consensus_code = consensus_codes[i]

In [63]:
#We first pick just enough validators if they are more than transactions
if len(mempool) < len(validators):
    validators = validators[:len(mempool)]
transaction_first_validator = math.ceil(len(mempool)/len(validators))
share_by_other_validators = math.floor(len(mempool)/len(validators))
divisions = []
divisions.append(transaction_first_validator)
for validator in validators[1:]:
    divisions.append(share_by_other_validators)

In [64]:
new_division = []
j = 0
for i in divisions:
    j += i
    new_division.append(j)

In [65]:
transaction_share = []
transaction_share.append(mempool[:transaction_first_validator])

In [66]:
for i in range(len(validators[1:])):
    transaction_share.append(mempool[new_division[i]-share_by_other_validators:new_division[i]])

In [67]:
#Staring the blockchain so that it runs
main_blockchain = Blockchain()

In [68]:
#Function to add blocks and validate_chain
def add_block_and_validate():
    for i in range(len(validators)):
        validator_copy = validators[i].chain
        validator_copy.add_block(transaction_share[i])
    if validator_copy.validate_chain():
        validator_copy.chain[-1].hash = consensus_codes[i]+str(validator_copy.chain[-1].hash)
        main_blockchain=validator_copy
    else:
        print(f"Validator index {i} is acting maliciously, there is need to intervene")

In [69]:
import time
start_time = time.time()
add_block_and_validate()
end_time = time.time()
time_taken = end_time - start_time
print(f"Time taken is: {time_taken} seconds")

Time taken is: 0.052541494369506836 seconds
